In [21]:
import os
import sys
import re
import pandas as pd
import csv
from typing import List
from functools import partial
sys.path.append("/home/kai/muku_esg/ESG-BERT/")
from allgrammer import grammer
from pdfminer.high_level import extract_text
%cd /home/kai/muku_esg/ESG-BERT/

/home/kai/muku_esg/ESG-BERT


In [22]:
class EsgBertLabel():
    def __init__(self, folder, csv_folder, word_cnt) -> None:
        self.folder = folder
        self.csv_folder = csv_folder
        self.word_cnt = word_cnt
    
    @staticmethod
    def ascii(text):
        text = re.sub(r'[^\x00-\x7F]', '', text)
        text = re.sub(r'[\x0C]', r"\n\n", text)
        return text
    
    @staticmethod
    def clean_text( x: str, folder, file_name):
        x = x.replace('\n', ' ')
        x = re.sub(r'[!\"#$%&\'()*\+,-.\/:;<=>?@\[\\\]^_`{|}~]', '', x)
        x = x.replace('\t', ' ')
        x = x.strip()
        if len(x.split()) < 8:
            return None
        else:
            return grammer(x, folder, file_name)
        
    def loop_folder(self, file_names: List[str]):
        for file_name in file_names:
            file_name = file_name.strip('.pdf')
            os.chdir('/home/kai/muku_esg/ESG-BERT')
            print(file_name)
            paragraphs = self.preprocess(file_name)
            self.predict(paragraphs)
            self.to_csv(paragraphs, file_name)

    def preprocess(self, file_name: str):     
        text = extract_text(f"./{self.folder}/{file_name}.pdf")
        text = ascii(text)
        text = text.strip('\"')
        paragraph = re.split(r'\n\s*\n', text)
        constant_preprocess = partial(self.clean_text, folder=self.folder, file_name=file_name)
        paragraph = map(constant_preprocess, paragraph)
        paragraph = list(paragraph)
        non_empty_paragraph = []
        for i in range(len(paragraph)):
            if paragraph[i] != None:
                x = paragraph[i].split()
                fifthlist = x[:300]
                fifthstr = ' '.join(fifthlist)
                non_empty_paragraph.append(fifthstr)
        return non_empty_paragraph
        
    def predict(self, paragraphs: List[str]):
        os.chdir('./predict_american')
        with open("./output.txt", "w") as file:
            file.truncate()
        with open("./paragraph.txt", "w") as file:
            file.truncate()
        for i in range(len(paragraphs)):
            # Open a file in write mode
            with open("./paragraph.txt", "w") as f:
                # Write the strings to the file
                f.writelines(paragraphs[i])
            os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt')
            os.system(f'echo >> output.txt')
        
    def to_csv(self, paragraphs: List[str], file_name: str):
        with open('./output.txt', 'r') as f:
            labels = f.readlines()
            labels = [x.strip('\n') for x in labels]
        ans = pd.DataFrame({'paragraph': paragraphs, 'predict_label': labels})
        ans.to_csv(f'../{self.csv_folder}/{self.word_cnt}/{file_name}.csv', sep=',', index=False, encoding='utf-8')

In [23]:
# 'american_pdf' or 'taiwan_pdf'
FOLDER = 'taiwan_pdf'
# 'predict_american' or 'predict_taiwan'
CSV_FOLDER = 'predict_taiwan'
# 'eight_new'
WORD_CNT = 'eight_new'

In [24]:
# Put desired pdf here
file_names = ['05_eme_1_46.pdf']

In [ ]:
esg_bert_label = EsgBertLabel(FOLDER, CSV_FOLDER, WORD_CNT)
esg_bert_label.loop_folder(file_names)